In [1]:
import argparse
import logging
from operator import mul
import time
import os

import pubweb.singlecell
from pubweb.hdf5 import Hdf5
from pubweb.commands.convert.singlecell.h5seurat import ImportH5Seurat
from pubweb.commands.validate.dimensions import ValidateDimensions
from pubweb.commands.annotate.geneid import AnnotateGeneId
from pubweb.commands.annotate.geneset import AnnotateGeneset
from pubweb.commands.export.lists import ExportLists
from pubweb.commands.export.attributes import ExportAttributes
from pubweb.commands.export.tables import ExportTables
from pubweb.commands.export.projections import ExportProjections
from pubweb.commands.export.spatial import ExportSpatial
from pubweb.commands.export.matrices import ExportMatrices
#from pubweb.commands.export.matrix_sparse import ExportMatrixSparse
#from pubweb.commands.export.matrix_dense import ExportMatrixDense
from pubweb.commands.summarize.genes import SummarizeGenes
from pubweb.commands.summarize.genemap import SummarizeGeneMap
from pubweb.commands.summarize.colors import SummarizeColors
from pubweb.commands.summarize.manifest import SummerizeManifest


/root/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import requests
import os
import h5py
from h5py import Dataset, Group, File
import pandas
import numpy as np
import boto3
import re
from io import BytesIO
from urllib.parse import urlparse
from shutil import copyfile
from pathlib import Path
from pubweb.enums import Attribute
from pubweb.hdf5 import Hdf5
from pubweb.commands.command import Command
import logging


In [3]:
logging.basicConfig(level='DEBUG')

In [4]:
datasetName='lee-diffexp'
fileToConvert = '/data/notebooks/input/lee_2020_processed.h5seurat'
outputFolder = '/data/notebooks/pubweb'
species = 'human'
overwriteHdf5 = True
python_wd = '/opt/pubweb'

In [5]:
data = h5py.File(fileToConvert, "r")

In [6]:
# h5seurat
outputFile = f'{outputFolder}/pubweb.hdf5'
if os.path.exists(outputFile) and overwriteHdf5:
    os.remove(outputFile)
hdf5 = Hdf5.load(outputFile, "a")

In [7]:
data['active.ident/levels']

<HDF5 dataset "levels": shape (28,), type "|O">

In [8]:
#%time hdf5 | ImportH5Seurat(fileToConvert, datasetName)

In [9]:
#hdf5.h5py['pubweb/lee-diffexp'].keys()

In [10]:
data['cell.names']

<HDF5 dataset "cell.names": shape (71509,), type "|O">

In [11]:
src = Hdf5.load(fileToConvert, "r")

# H5seurat.py

In [12]:
dataset = f'/pubweb/{datasetName}'

In [13]:
pconf = {
    "cell": "cell.names",
    "reductions/pca/cell.embeddings": {"type": "projection",
                                       "col": "reductions/pca/features"
                                      },
    "reductions/pca/features": {"gene": 1},
    "reductions/ref.spca/cell.embeddings": {"type": "projection",
                                            "row": "cell.names"},
    "reductions/ref.umap/cell.embeddings": {"type": "projection",
                                            "row": "cell.names"},
    "assays/RNA/counts": {"type": "matrix"},
    "assays/RNA/data": {"type": "matrix"},
    "assays/SCT/counts": {"type": "matrix"},
    "assays/SCT/data": {"type": "matrix"},
    "assays/predicted_ADT/data": {"type": "matrix"},
    "assays/prediction.score.celltype.l1/data": {"type": "matrix"},
    "assays/prediction.score.celltype.l2/data": {"type": "matrix"},
    "assays/prediction.score.celltype.l3/data": {"type": "matrix"},
    "misc/merged_umap": {"type": "matrix"},
    "graphs/rna.snn": {"type": "matrix"}
}

In [14]:
group = hdf5.h5py.create_group(f'{dataset}/lists')
group = hdf5.h5py.create_group(f'{dataset}/tables')
group = hdf5.h5py.create_group(f'{dataset}/matrix')
group = hdf5.h5py.create_group(f'{dataset}/projections')

In [15]:
def copy_table(src, dest, table, dataset, rows_dataset=None, dest_prefix='tables', is_gene=False):
    print(f'Copying table {table.name}')
    dim_count = 1 if len (table.shape) == 1 else table.shape[1]
    # figure out the names
    name_bits = [ x for x in table.name.split('/') if len(x) > 0]
    target_name = name_bits[-1].replace('.','_')
    parent_name = "_".join(name_bits[:-1]).replace('.','_')
    col_headers = np.array([f"{os.path.basename(table.name)}-{n}" for n in range(1, dim_count + 1)])
    # make a new group and copy
    target_path = f'{dataset}/{dest_prefix}/{parent_name}/{target_name}'
    group = dest.h5py.create_group(target_path)
    dest.h5py.create_dataset(f'{target_path}/columns', data=col_headers.astype("S10"), dtype='S10')
    src.h5py.copy(table.name, group, name="values")
    # add the rows dataset in some form
    if rows_dataset is not None:
        src.h5py.copy(rows_dataset, group, name="rows")
    else:
        rowcount = len(table[:]) / len(col_headers)
        print(f"There are {rowcount} rows")
        row_headers = np.arange(rowcount)
        dest.h5py.create_dataset(f'{target_path}/rows', data=row_headers.astype("S10"), dtype='S10')
    if is_gene:
        print(f"Setting PW:Gene for {target_path}/{table.name}")
        dest[f'{target_path}/{table.name}'].attrs.create("PW:Gene", "")

In [16]:
def copy_dense_matrix(src, dest, table_name, dataset):
    # naming
    name_bits = [ x for x in table_name.split('/') if len(x) > 0]
    target_name = name_bits[-1].replace('.','_')
    parent_name = "_".join(name_bits[:-1]).replace('.','_')
    print(f"Copying dense matrix from {table_name} to /matrix/{parent_name}")
    # copy
    # note, need to trim the leading '/' off to use getGroupsWithName
    if len(dest.getGroupsWithName(f"{dataset}/matrix/{parent_name}"[1:])) < 1:
        group = dest.h5py.create_group(f'{dataset}/matrix/{parent_name}')
    src.h5py.copy(table_name, dest[f"{dataset}/matrix/{parent_name}"])
    

In [17]:
def copy_sparse_matrix(src, dest, group_name, dataset):
    print(f'Copying sparse matrix from {group_name}')
    target_name = group_name.replace('/','_')
    # make a new group and copy
    group = dest.h5py.create_group(f'{dataset}/matrix/{target_name}')
    src.h5py.copy(f"{group_name}/data", group)
    src.h5py.copy(f"{group_name}/indices", group)
    src.h5py.copy(f"{group_name}/indptr", group)

In [18]:
def copy_projection(src, dest, table_name, dataset, rows_dataset=None, cols_dataset=None):
    # naming and headers
    projection_name = table_name.replace('ref.', '').replace('.','_').replace('/','.')
    dim_count = src[table_name].shape[0]
    target_path = f'{dataset}/projections/{projection_name}/{dim_count}'
    if cols_dataset is None:
        cols_dataset = np.array([f"{os.path.basename(table_name)}-{n}" for n in range(1, dim_count + 1)])
    # now copy everything
    group = dest.h5py.create_group(target_path)
    dest.h5py.create_dataset(f'{target_path}/columns', data=cols_dataset.astype("S10"), dtype='S10')
    src.h5py.copy(f'{table_name}', group, name="values")
    if rows_dataset is not None:
        src.h5py.copy(rows_dataset, group, name="rows")
    else:
        rowcount = len(src[table_name][:]) / len(cols_dataset)
        print(f"There are {rowcount} rows")
        row_headers = np.arange(rowcount)
        dest.h5py.create_dataset(f'{target_path}/rows', data=row_headers.astype("S10"), dtype='S10')

In [19]:
def copy_group(src, dest, group_name, conf):
    # get a list of datasets
    group_conf = conf.get(group_name,{})
    
    current_datasets = src.getDatasetsWithPath(group_name)
    current_dataset_bases = [x.replace(f"{group_name}/", '') for x in current_datasets]
    
    # sparse matrices are a special case
    if group_conf.get('type') == 'matrix':
        is_sparse = all(elem in current_dataset_bases for elem in ["data", "indices", "indptr"])
        if is_sparse:
            print(f'sparse copy from {group_name}/*')
            copy_sparse_matrix(src, dest, group_name, dataset)
    else:
        for ds, ds_short in zip(current_datasets, current_dataset_bases):
            # figure out what applies to this table
            table_conf = conf.get(ds, {"type": "table"})
            action = table_conf.get('type')
            print(f"Dataset '{ds}' has action={action}")
        # Do different copies depending on the action
            if action == 'projection':
                rows, cols = None, None
                if table_conf.get('row') is not None:
                    rows = src[table_conf.get('row')]
                if table_conf.get('col') is not None:
                    cols = src[table_conf.get('col')][:]
                print(f"copy_projection for {ds} has rows={rows} & cols={cols}")
                copy_projection(src=src,
                                dest=dest,
                                table_name=ds, 
                                dataset=dataset,
                                rows_dataset=rows,
                                cols_dataset=cols)   
            elif action == 'matrix':
                print(f"copy matrix {ds}")
                copy_dense_matrix(src, dest, ds, dataset)
            if action == 'table':
                is_gene = True if table_conf.get('gene',0) == 1 else False
                copy_table(src=src, dest=dest, table=src[ds], dataset=dataset, is_gene=is_gene)

    # now recurse!
    child_groups = src.getGroupsWithPath(group_name)
    for child in child_groups:
        #print(f"Recursing down to {child}")
        copy_group(src, dest, child, conf)

In [ ]:
for k in src.h5py['/'].keys():
    print(f"Copying top-level group {k}")
    if isinstance(src[k], Dataset):
        copy_table(src, dest=hdf5, table=src[k], dataset=dataset)
    else:
        copy_group(src=src, dest=hdf5, group_name=k, conf=pconf)

DEBUG:h5py._conv:Creating converter from 3 to 5


Copying top-level group active.ident
Dataset 'active.ident/levels' has action=table
Copying table /active.ident/levels
There are 28.0 rows
Dataset 'active.ident/values' has action=table
Copying table /active.ident/values
There are 71509.0 rows
Copying top-level group assays
Dataset 'assays/RNA/features' has action=table
Copying table /assays/RNA/features
There are 22221.0 rows
sparse copy from assays/RNA/counts/*
Copying sparse matrix from assays/RNA/counts
sparse copy from assays/RNA/data/*
Copying sparse matrix from assays/RNA/data
Dataset 'assays/SCT/features' has action=table
Copying table /assays/SCT/features
There are 22160.0 rows
Dataset 'assays/SCT/scale.data' has action=table
Copying table /assays/SCT/scale.data


In [ ]:
%time hdf5 | AnnotateGeneId(species=species)

In [ ]:
#%time hdf5 | ExportProjections(outputFolder)

In [ ]:
%time hdf5 | ExportTables(outputFolder)

In [ ]:
#%time hdf5 | ExportMatrices(outputFolder)